In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import cython
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle

In [2]:
g_ket = basis(5,0)
u_ket = basis(5,1)
x_ket = basis(5,2)
a_ket = basis(5,3)
b_ket = basis(5,4)

g_bra = g_ket.dag()
u_bra = u_ket.dag()
x_bra = x_ket.dag()
a_bra = a_ket.dag()
b_bra = b_ket.dag()

g_id = g_ket*g_bra
u_id = u_ket*u_bra
x_id = x_ket*x_bra
a_id = a_ket*a_bra
b_id = b_ket*b_bra

gx_swap = x_ket*g_bra
ux_swap = x_ket*u_bra

spont_decay = b_ket*x_bra
cav_decay = a_ket*g_bra

In [3]:
gamma = 2*np.pi
kappas = [0.01,0.1,1]

NUM_COOPS = 25
NUM_TIMES = 25
STEPS_PER_SEC = 1000

tophat_pulse = 'np.piecewise(t, [t<T], [omega, 0])'
opts = Options(rhs_reuse=True)

current_kappa = kappas[0]

In [4]:
def save_obj(object, name):
    with open(name+'.pickle', 'wb') as f:
        print(name+'.pickle')
        pickle.dump(object, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [5]:
total_cores = multiprocessing.cpu_count()
if total_cores > 60:
    num_cores = 60
else:
    num_cores = total_cores

for kappa in kappas:
    print('yeet')

    gmin = np.sqrt(gamma/(2*kappa))
    gmax = 10*np.sqrt(gamma/(2*kappa))
    g_ratios = [np.sqrt(0.5**2*gamma/(2*kappa))] + np.logspace(np.log10(gmin), np.log10(gmax), NUM_COOPS)

    t_min = 1
    t_max = 2*np.log10(2)
    times = np.logspace(t_min, t_max, NUM_TIMES)

    current_time = times[0]

    def find_efficiency(g_ratio, omega, delta, pulse_time, psi0=u_ket, pulse_shape='tophat', added_time=1):
        
        time = pulse_time + added_time

        omega *= 2*np.pi
        delta *= 2*np.pi
        g0 = g_ratio*gamma

        H_args = {
            'omega': omega,
            'delta': delta,
            'g0': g0,
            'T': pulse_time,
            'wSTIRAP': np.pi/pulse_time
        }

        global current_time
        global current_kappa
        
        if time != current_time or kappa != current_kappa:
            rhs_clear()
            opts.rhs_reuse = False
            current_time = time
            current_kappa = kappa

        # define time steps
        runtime = time/gamma
        num_steps = round(STEPS_PER_SEC*runtime + 1)
        if num_steps < 1001:
            num_steps = 1001
        
        t_res = 2
        int_time = round(runtime, t_res)*10**t_res
        t = np.linspace(0, int_time, num_steps)
        t /= 10**t_res

        H0 = -(gx_swap + gx_swap.dag())
        H1 = -1/2*(ux_swap + ux_swap.dag())
        H2 = g_id + u_id

        if pulse_shape == 'sin':
            pulse = 'omega*np.sin(wSTIRAP*t)**2'
        elif pulse_shape == 'tophat':
            pulse = 'omega'

        H=[[H0,'g0'], [H1, pulse], [H2, 'delta']]

        c_ops = [np.sqrt(gamma/2)*spont_decay, np.sqrt(kappa*gamma/2)*cav_decay]

        result = mesolve(H, u_ket, t[], c_ops, args=H_args, options=opts)
        opts.rhs_reuse = True
        # print(expect(a_id, result.states[-1]))
        return expect(a_id, result.states[-1])
    

    def iterative_optimiser_2D(inputs):
        g_ratio, omega_0, delta_0, time = inputs

        omg_res = [1000, 300, 100, 30, 10, 3, 1, 0.3, 0.1]
        delt_res = [1000, 300, 100, 30, 10, 3, 1, 0.3, 0.1]


        initial_eff = find_efficiency(g_ratio, omega_0, delta_0, time)
    
        def find_eff(omega,delta):
            return find_efficiency(g_ratio, omega, delta, time)

        for res in omg_res:
            # print(f'res = {res}')
            

            finished = False
            new_step = 0
            

            while not finished:
                print(omega_0, delta_0, initial_eff)


                if new_step == 0:
                    temp_effs = [initial_eff, find_eff(omega_0-res, delta_0), find_eff(omega_0 + res, delta_0)]
                elif new_step == 1:
                    temp_effs = [initial_eff, temp_effs[0],find_eff(omega_0+res, delta_0)]
                else:
                    temp_effs = [initial_eff, find_eff(omega_0-res,delta_0), temp_effs[0]]
                
                initial_eff = max(temp_effs)
                if np.argmax(temp_effs) == 0:
                    
                    finished = True
                elif np.argmax(temp_effs) == 1:
                    new_step = -1
                    omega_0 -= res
                else:
                    new_step = 1
                    omega_0 += res

        if omega_0 < 0:
            omega_0 *= -1

        for res in delt_res:
            # print(f'res = {res}')
            

            finished = False
            new_step = 0
            

            while not finished:
                print(omega_0, delta_0, initial_eff)


                if new_step == 0:
                    temp_effs = [initial_eff, find_eff(omega_0, delta_0-res), find_eff(omega_0, delta_0+res)]
                elif new_step == 1:
                    temp_effs = [initial_eff, temp_effs[0],find_eff(omega_0, delta_0+res)]
                else:
                    temp_effs = [initial_eff, find_eff(omega_0,delta_0-res), temp_effs[0]]
                
                initial_eff = max(temp_effs)
                if np.argmax(temp_effs) == 0:
                    
                    finished = True
                elif np.argmax(temp_effs) == 1:
                    new_step = -1
                    delta_0 -= res
                else:
                    new_step = 1
                    delta_0 += res

        # print('this is where the fun begins')

        delta_res = [3,1,0.3, 0.1]
        omg_res = [3,1,0.3,0.1]

        def new_temp_eff(coords, temp_effs, omega_0, delta_0, d_omg, d_del):
            new_effs = [[0,0,0],[0,0,0],[0,0,0]]

            for x in range(-1,2):
                for y in range(-1,2):

                    if 3> coords[0] + y >= 0 and 3> coords[1] + x >= 0:
    
                        new_effs[y+1][x+1] = temp_effs[coords[0] + y][coords[1] + x]
                    else:

                        new_effs[y+1][x+1] = find_eff(omega_0 + y*d_omg, delta_0 + x*d_del)

            return new_effs

        print('this is where the fun begins')

        for k in range(len(delta_res)):

            finished = False
            new_step = 0

            d_omg = omg_res[k]
            d_del = delta_res[k]

            temp_effs = [
                [find_eff(omega_0-d_omg,delta_0-d_del), find_eff(omega_0-d_omg,delta_0), find_eff(omega_0-d_omg,delta_0+d_del)],
                [find_eff(omega_0,delta_0-d_del), find_eff(omega_0,delta_0), find_eff(omega_0,delta_0+d_del)],
                [find_eff(omega_0+d_omg,delta_0-d_del), find_eff(omega_0+d_omg,delta_0), find_eff(omega_0+d_omg,delta_0+d_del)]
            ]
            # print(temp_effs)
            max_args = [np.argmax(sub_temp_effs) for sub_temp_effs in temp_effs]
            max_effs = [max(sub_temp_effs) for sub_temp_effs in temp_effs]

            max_coords = [np.argmax(max_effs),max_args[np.argmax(max_effs)]]
            # print(max_coords)
            initial_eff = max(max_effs)

            if temp_effs[1][1] == initial_eff:
                max_coords = [1,1]

            while not finished:
                print(omega_0, delta_0, initial_eff)

                if max_coords == [1,1]:
                    finished = True

                
                else:
                    delta_0 += (max_coords[1] - 1)*d_del
                    omega_0 += (max_coords[0] - 1)*d_omg

                    # print(temp_effs)
                    # print(max_coords, delta_0, omega_0, initial_eff)


                    temp_effs = new_temp_eff(max_coords, temp_effs, omega_0, delta_0, d_omg, d_del)

                    max_args = [np.argmax(sub_temp_effs) for sub_temp_effs in temp_effs]
                    max_effs = [max(sub_temp_effs) for sub_temp_effs in temp_effs]

                    # maxes = [max(w) for w in v]
                    # arg_maxes = [np.argmax(w) for w in v]

                    # arg_max = np.argmax(maxes)
                    # maxy = v[arg_max][arg_maxes[arg_max]]

                    max_coords = [np.argmax(max_effs), max_args[np.argmax(max_effs)]]
                    initial_eff = max(max_effs)
                        
        return {
            'g_ratio': g_ratio,
            'time': time,
            'omega': np.abs(omega_0),
            'delta': np.abs(delta_0),
            'efficiency': initial_eff
        }

    print(iterative_optimiser_2D([g_ratios[-5], 100,0,times[8]]))
    print(iterative_optimiser_2D([g_ratios[-4], 100,0,times[8]]))
    print(iterative_optimiser_2D([g_ratios[-3], 100,0,times[8]]))
    print(iterative_optimiser_2D([g_ratios[-2], 100,0,times[8]]))
    print(iterative_optimiser_2D([g_ratios[-1], 100,0,times[8]]))
    first_time = True
    input_args = []
    output_list = []
    for j in range(len(times)):
        input_args.append([[g_ratios[k], 100, 0, times[8]] for k in range(len(g_ratios))])

    for input_set in input_args:

        if not first_time:
            for j in range(len(times)):
                input_args[j][1] = processed_list[j]['omega']
                input_args[j][2] = processed_list[j]['delta']

        if __name__ == "__main__":
            inputs = tqdm(input_set)
            processed_list = Parallel(n_jobs=num_cores)(delayed(iterative_optimiser_2D)(i) for i in inputs)
            output_list.append(processed_list)
            first_run = False
    
    save_obj(output_list, f'2D tophat time variance kappa = {kappa}gamma.pickle')



yeet
100 0 0.005682987675895387
100 0 0.005682987675895387
-200 0 0.011742785328102039
-200 0 0.011742785328102039
-300 0 0.01224663438301034
-300 0 0.01224663438301034
-270 0 0.01249774029118593
-270 0 0.01249774029118593
-260 0 0.012516076639060306
-260 0 0.012516076639060306
-257 0 0.012524916148001965
-257 0 0.012524916148001965
-257 0 0.012524916148001965
-257 0 0.012524916148001965
257 0 0.012524916148001965
257 -1000 0.01950582004821933
257 -2000 0.020362489438065654
257 -3000 0.020733717923057143
257 -3000 0.020733717923057143
257 -3000 0.020733717923057143
257 -3000 0.020733717923057143
257 -3000 0.020733717923057143
257 -2990 0.020736626303180353
257 -2990 0.020736626303180353
257 -2993 0.02073790976006074
257 -2993 0.02073790976006074
257 -2994 0.020737925191408817
257 -2994 0.020737925191408817
257 -2993.7 0.020737941818489605
257 -2993.7 0.020737941818489605
257 -2993.6 0.020737943229016283
this is where the fun begins
257 -2993.6 0.020737943229016283
257 -2993.6 0.0207379